In [1]:
import os
import torch
import pickle as pkl
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_utils import (
    results2df,
)
from plot_utils import plot_behavior_distribution
from sklearn.dummy import DummyClassifier
from torchmetrics.functional.classification import (
    multilabel_average_precision,
    multilabel_f1_score,
)
from matplotlib.colors import rgb2hex

**Data Loading**

In [2]:
metadata_file = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/metadata/with_negative_pairing/new_metadata.csv"
behaviours_file = "../dataset/metadata/behaviours.txt"
segments_file = "../dataset/metadata/segments.txt"


metadata_df = pd.read_csv(metadata_file)

with open(behaviours_file, "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

with open(segments_file, "rb") as f:
    segments = [seg.decode("utf-8").strip() for seg in f.readlines()]

dummy_aps = [
    0.0235,
    0.0531,
    0.1467,
    0.1292,
    0.0346,
    0.2582,
    0.0776,
    0.1038,
    0.0155,
    0.0115,
    0.4553,
    0.1002,
    0.5724,
    0.0563,
]

exc_dummy_aps = [
    0.0206,
    0.0453,
    0.1432,
    0.1658,
    0.0227,
    0.2533,
    0.0618,
    0.1267,
    0.0175,
    0.0062,
    0.4027,
    0.1432,
    0.62,
    0.0494,
]

In [3]:
def calculate_metrics(df, round_to=3, show_per_class=True):
    map_values = multilabel_average_precision(
        torch.tensor(np.stack(df["pred"])),
        torch.tensor(np.stack(df["label"])),
        num_labels=14,
        average="none",
        # thresholds=200,
    )
    avg_map = round(map_values.mean().item(), round_to)
    if show_per_class:
        map_values_list = []
        for v in map_values:
            map_v = round(v.item(), round_to)
            map_values_list.append(map_v)

        return map_values_list
    return avg_map

In [4]:
val_results = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/results/paper_results"

results_df = None

# Get paths to all pickle files
val_paths = []
for root, dirs, files in os.walk(val_results):
    for file in files:
        if file.endswith(".pkl"):
            val_paths.append(os.path.join(root, file))

for val_path in val_paths:

    col_name = val_path.split("/")[-2]
    with open(
        val_path,
        "rb",
    ) as f:
        val_data = pkl.load(f)

    train_df, val_df = results2df(
        val_data, val_data, metadata_df, right_on="subject_id_fg"
    )

    if results_df is None:
        results_df = pd.DataFrame(
            {
                "behaviour": behaviours,
                "segment": segments,
                f"{col_name}": calculate_metrics(val_df),
            }
        )
    else:
        results_df[col_name] = calculate_metrics(val_df)

results_df["dummy_aps"] = dummy_aps
results_df["exc_dummy_aps"] = exc_dummy_aps

/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/notebooks/data_utils.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))
/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/notebooks/data_utils.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))
/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/notebooks/data_utils.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df["pred"]

In [5]:
results_df.columns

Index(['behaviour', 'segment',
       'model=slow_r50_ds=panaf_seq_fgbg_rand_sub_dup_bg',
       'model=slow_r50_baseline_cropped_timestamp',
       'model=slow_r50_exclusive_utms_bg_only',
       'model=slow_r50_bg_only_rm-bgs',
       'model=slow_r50_e200_fg_minus_bg_sched=linear_agg=conv',
       'model=slow_r50_bg-only_rm-bgs',
       'model=slow_r50_exclusive_utms_fg_only',
       'model=slow_r50_e200_fg_minus_bg_sched=linear_agg=max_pool',
       'model=slow_r50_ds=panaf_seq_fgbg_rand_sub',
       'model=slow_r50_e200_fg_minus_bg_ft',
       'model=slow_r50_ds=panaf_seq_fgbg_rand_sub_full_bg_bg2',
       'model=slow_r50_exclusive_utms_fg_only_test',
       'model=r50_fg_minus_bg_e200_sched=exp_agg=max_pool',
       'model=slow_r50_exclusive_utms_bg_only_rm-bgs',
       'model=slow_r50_e200_fg_minus_bg_sched=linear_agg=avg_pool',
       'model=slow_r50_exclusive_utms_fg_only_train',
       'model=r50_fg_minus_bg_e200_sched=exp_agg=avg_pool',
       'model=slow_r50_ds=panaf_seq_fg_

In [ ]:
results_df[
    [
        "behaviour",
        "segment",
        "model=slow_r50_ds=panaf_seq_fgbg_rand_sub",
        "model=slow_r50_ds=panaf_seq_fgbg_rand_sub_dup_bg",
        "model=slow_r50_ds=panaf_seq_fgbg_rand_sub_full_bg",
    ]
]

**Global results**

In [7]:
global_results_df = results_df[
    [
        "behaviour",
        "segment",
        "dummy_aps",
        "model=slow_r50_e200_baseline_results",
        "model=slow_r50_ds=panaf_seq_fgbg_add-sub-global",
        "model=slow_r50_ds=panaf_seq_fgbg_add-global",
    ]
]

In [ ]:
global_results_df

In [ ]:
global_results_df[global_results_df.columns[2:]].mean()

In [ ]:
store = []
for s in global_results_df.segment.unique():
    dummy_perfomance = global_results_df[global_results_df.segment == s][
        "dummy_aps"
    ].mean()
    baseline_performance = global_results_df[global_results_df.segment == s][
        "model=slow_r50_e200_baseline_results"
    ].mean()
    fixed_sub_mean_add = global_results_df[global_results_df.segment == s][
        "model=slow_r50_ds=panaf_seq_fgbg_add-global"
    ].mean()
    mean_sub_mean_add = global_results_df[global_results_df.segment == s][
        "model=slow_r50_ds=panaf_seq_fgbg_add-sub-global"
    ].mean()

    store.append(
        {
            "segment": s,
            "dummy_aps": dummy_perfomance,
            "baseline": baseline_performance,
            "fixed_sub_mean_add": fixed_sub_mean_add,
            "mean_sub_mean_add": mean_sub_mean_add,
        }
    )
global_results_s_df = pd.DataFrame(store)
global_results_s_df

**Exclusive results**

In [11]:
exc_results_df = results_df[
    [
        "behaviour",
        "segment",
        "exc_dummy_aps",
        "model=slow_r50_exclusive_utms_bg_only_rm-bgs",
        "model=slow_r50_exclusive_utms_fg_only",
        "model=slow_r50_exclusive_utms_fg-bg-lambda",
    ]
]

In [ ]:
exc_results_df

In [ ]:
exc_results_df[exc_results_df.columns[2:]].mean()

In [ ]:
store = []
for s in exc_results_df.segment.unique():
    dummy_perfomance = exc_results_df[exc_results_df.segment == s][
        "exc_dummy_aps"
    ].mean()
    baseline_performance = exc_results_df[exc_results_df.segment == s][
        "model=slow_r50_exclusive_utms_bg_only_rm-bgs"
    ].mean()
    fixed_sub_mean_add = exc_results_df[exc_results_df.segment == s][
        "model=slow_r50_exclusive_utms_fg_only"
    ].mean()
    mean_sub_mean_add = exc_results_df[exc_results_df.segment == s][
        "model=slow_r50_exclusive_utms_fg-bg-lambda"
    ].mean()

    store.append(
        {
            "segment": s,
            "exc_dummy_aps": dummy_perfomance,
            "exclusive_utms_bg_only": baseline_performance,
            "exclusive_utms_fg_only": fixed_sub_mean_add,
            "exclusive_utms_fg-bg-lambda": mean_sub_mean_add,
        }
    )
exc_results_s_df = pd.DataFrame(store)
exc_results_s_df

**FG-BG**

In [ ]:
results_df.columns

In [ ]:
results_df[
    [
        "model=r50_fg_minus_bg_e200_sched=exp_agg=max_pool",
        "model=r50_fg_minus_bg_e200_sched=exp_agg=conv",
        "model=r50_fg_minus_bg_e200_sched=exp_agg=avg_pool",
    ]
]

In [7]:
results_df[
    [
        # "behaviour",
        # "segment",
        # "dummy_aps",
        "model=slow_r50_bg_only_rm-bgs",
        "model=slow_r50_e200_baseline_results",
        # "model=slow_r50_ds=panaf_seq_fg_minus_bg",
        # "model=slow_r50_e200_fg_minus_bg_sched=linear_agg=avg_pool",
        # "fg-fixed_bg+bg2",
        # "model=slow_r50_e200_fg_minus_bg_ft",
    ]
]

,model=slow_r50_bg_only_rm-bgs,model=slow_r50_e200_baseline_results
0,0.200,0.409
1,0.189,0.329
2,0.447,0.448
3,0.595,0.696
4,0.317,0.563
5,0.633,0.840
6,0.183,0.211
7,0.358,0.342
8,0.213,0.231
9,0.211,0.040


In [18]:
bg_df = results_df[
    [
        "behaviour",
        "dummy_aps",
        "model=slow_r50_e300_bg-only",
        "model=slow_r50_e200_baseline_results",
    ]
]

In [ ]:
bg_df["dummy_bg_diff"] = bg_df["model=slow_r50_e300_bg-only"] - bg_df["dummy_aps"]
bg_df["baseline_bg_diff"] = (
    bg_df["model=slow_r50_e200_baseline_results"] - bg_df["model=slow_r50_e300_bg-only"]
)

In [ ]:
bg_df.sort_values(by=["dummy_bg_diff"], ascending=False)